In [16]:
import pandas as pd
import os
from time import sleep
from collections import defaultdict  #to store multiple values of a column
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait

In [21]:
class CaseStatus():
    def __init__(self,url):
        self.url = url

    #--Driver setup--#
        try:
            print('Preparing Driver setup...')
            firefox_driver_path = 'C:/Users/HP/Anaconda3/envs/beautifulsoup/geckodriver.exe'
            self.driver = webdriver.Firefox(executable_path=firefox_driver_path)

    #--Opening Website--#
        
            print('Opening URL...')
            self.driver.get(url)
            WebDriverWait(self.driver,3)
            
        except Exception as e:
            print(e)
            print('Closing driver')
            self.driver.close()
            exit()
            
        
#----Entering credentials----#            
    def get_data(self,diary_number=None, year=None): 
            print('Entering credentials...\n')

    #--Diary number input--#
            diary_number_holder = self.driver.find_element_by_id('CaseDiaryNumber')
            diary_number_holder.send_keys(diary_number)

    #---Year input---#
            year_holder = self.driver.find_element_by_id('CaseDiaryYear')
            for option in year_holder.find_elements_by_tag_name('option'):
                if option.text == str(year):
                    option.click()
                    break


    #--Captcha input--#
            captcha_text = self.driver.find_element_by_id('cap')
            captcha_text = captcha_text.text

            captcha_holder = self.driver.find_element_by_id('ansCaptcha')
            captcha_holder.send_keys(int(captcha_text))

    #---Submit button---#
            self.driver.find_element_by_id('getCaseDiary').click()
            sleep(3)

    #--Locating required element--#
            try:
                table_html_content = self.driver.find_element_by_id('DNdisplay')
            except Exception as e:
                print(e)
            else:
                header_list = table_html_content.find_elements_by_tag_name('h5')
                print('Table Title:')
                for header in header_list:
                    print(header.text)
                print()

    #--Scrapping Table--#
            table = None
            try:
                table_content = table_html_content.find_elements_by_tag_name('tr')
                if table_content[0].text == '':
                    print(table_html_content.find_element_by_tag_name('p').text,'\n')
                else:
                    print('Scrapping Table...')
                    table = defaultdict(list)

            #--Separating colunm names--#
                    column_names = []
                    column_content = table_html_content.find_elements_by_tag_name('td')
                    for index,column in enumerate(column_content):
                        if index%2==0:
                            column_names.append(column.text)
                    column_names = column_names[:-2]

            #--Separting records--#
                    for index in range(len(column_names)):
                        column = table_content[index].text[:len(column_names[index])]
                        record = table_content[index].text[len(column_names[index]):]

                        if record == '':
                            record = 'None'

                        if '\n' in record:
                            table[column].append(record.replace('\n',''))
                        else:
                            table[column].append(record)
            except Exception as e:
                table = None
                print(e)



    #--Dataframe--#
            df = pd.DataFrame.from_dict(table)
            if table == None:
                print('Could not scrape the table')
            else:
                print('Saving table...\n')
                #current_script_path = os.path.dirname(os.path.abspath(__file__))
                #df.to_csv(os.path.join(current_script_path, "Case-Status.csv"))
                df.to_csv(os.getcwd() + r'\Case-Status.csv',index=False)
                #df.to_csv(path_or_buf=., index=False)


            print('Closing driver')
            self.driver.close()
            return df


In [22]:
if __name__ == '__main__':
    base_url = 'https://main.sci.gov.in/case-status'
    try:
        diary_no = int(input('Enter Diary Number within 1 to 100= '))
        year = int(input('Enter year within 1950 to 2020= '))
        print()
    except Exception as e:
        print(e)
    else:
        if (diary_no>=1 and diary_no<=100) and (year>=1950 and year<=2020):
            obj = CaseStatus(base_url)
            dataframe = obj.get_data(diary_no,year)
        else:
            print('Enter credentials within range')

Enter Diary Number within 1 to 100= 2
Enter year within 1950 to 2020= 2020

Preparing Driver setup...
Opening URL...
Entering credentials...

Table Title:
Diary No.- 2 - 2020
CITY MONTESSORI SCHOOL vs. ASHOK BHARGAVA

Scrapping Table...
Saving table...

Closing driver


In [23]:
#dataframe
dataframe

,Diary No.,Case No.,Present/Last Listed On,Status/Stage,Tentatively case may be listed on (likely to be listed on),Category,Act,Petitioner(s),Respondent(s),Pet. Advocate(s),Resp. Advocate(s),U/Section
0,2/2020 Filed on 02-01-2020 10:37 AMPENDING [...,MA 002165 - / 2020 Registered on 26-11-2020,20-11-2020 [HON'BLE MR. JUSTICE UDAY UMESH LA...,Pending - (Motion Hearing[AFTER NOTICE (FOR A...,04-01-2021 (Computer generated),1703-Contempt Of Court Matters : Other civil ...,None,1 CITY MONTESSORI SCHOOL FOUNDER MANAGER JA...,1 ASHOK BHARGAVA S/D/W/Thru:- LATE N.K. BHAR...,R. P. GUPTA,None,None
